In [8]:
from Function import Function
import numpy as np
import pandas as pd
from Module import Lineaire, MSE, Hinge
from tools import get_patches, pad_to_32
import torch
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import sys
%load_ext autoreload
%autoreload 2

from load_data import get_train_test_loaders

/home/samutamm/virtualenvironments/datascience/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/samutamm/virtualenvironments/datascience/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
train_loader,test_loader = get_train_test_loaders()

In [9]:
def get_minibatches(loader):
    for data, target in loader:
        batch_n = data.size()[0]
        X = data.view(-1, batch_n)
        X = torch.cat((torch.ones((1,batch_n)), X), 0)
        
        y = target.view(batch_n, 1).float()
        yield X, y

In [ ]:
torch.cuda.set_device(0)

layers_dims = [28*28 + 1, 10, 1]

epsilon = 0.0001

l1 = Lineaire(layers_dims[0], layers_dims[1])
a1 = None # Tanh
l2 = Lineaire(layers_dims[1], layers_dims[2])
a2 = None # Sigmoid
loss = MSE()

l1.initialize_parameters()

train_errors = []
test_errors = []



for epoch in range(40):
    if epoch % 10 == 0:
        print("Epoch {}".format(str(epoch)))
    train_error = 0
    for i,(X, y) in enumerate(get_minibatches(train_loader)):
        model.grad_zero()      
        
        z1 = model.forward(X)
        yhat = torch.sign(z1)
        train_error += loss.forward(y,yhat).sum()
        
        #import pdb; pdb.set_trace()
        
        delta_Z = loss.backward(y,z1)
        model.backward_update_gradient(X, delta_Z)
        
        model.update_parameters(epsilon)